In [1]:
import pandas as pd
import plotly.express as px
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix

## Carregar os dados

In [2]:
df_leads = pd.read_csv('../bagging/aula/leads_cleaned.csv')

In [3]:
df_leads.head()

,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Search,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,A free copy of Mastering The Interview,Last Notable Activity
0,API,Olark Chat,0,0,0,0.0,0,0.0,Page Visited on Website,0,0,0,0,0,0,0,Modified
1,API,Organic Search,0,0,0,5.0,674,2.5,Email Opened,0,0,0,0,0,0,0,Email Opened
2,Landing Page Submission,Direct Traffic,0,0,1,2.0,1532,2.0,Email Opened,0,0,0,0,0,0,1,Email Opened
3,Landing Page Submission,Direct Traffic,0,0,0,1.0,305,1.0,Unreachable,0,0,0,0,0,0,0,Modified
4,Landing Page Submission,Google,0,0,1,2.0,1428,1.0,Converted to Lead,0,0,0,0,0,0,0,Modified


In [4]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9074 entries, 0 to 9073
Data columns (total 17 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Lead Origin                             9074 non-null   object 
 1   Lead Source                             9074 non-null   object 
 2   Do Not Email                            9074 non-null   int64  
 3   Do Not Call                             9074 non-null   int64  
 4   Converted                               9074 non-null   int64  
 5   TotalVisits                             9074 non-null   float64
 6   Total Time Spent on Website             9074 non-null   int64  
 7   Page Views Per Visit                    9074 non-null   float64
 8   Last Activity                           9074 non-null   object 
 9   Search                                  9074 non-null   int64  
 10  Newspaper Article                       9074 non-null   int6

## Preparação dos dados

In [5]:
X = df_leads.drop(columns=['Converted'], axis=1)
y = df_leads['Converted']

In [6]:
import joblib

preprocessor = joblib.load('../bagging/aula/preprocessor_dataset_leads.pkl')

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=51)

# Aplicar preprocessor
X_train = preprocessor.fit_transform(X_train).toarray()
X_test = preprocessor.transform(X_test).toarray()

In [8]:
print(f'Conjunto de Treinamento: {X_train.shape}')
print(f'Conjunto de Teste: {X_test.shape}')

Conjunto de Treinamento: (7259, 68)
Conjunto de Teste: (1815, 68)


In [9]:
preprocessor.get_feature_names_out()

array(['num__Do Not Email', 'num__Do Not Call', 'num__TotalVisits',
       'num__Total Time Spent on Website', 'num__Page Views Per Visit',
       'num__Search', 'num__Newspaper Article', 'num__X Education Forums',
       'num__Newspaper', 'num__Digital Advertisement',
       'num__Through Recommendations',
       'num__A free copy of Mastering The Interview',
       'cat__Lead Origin_API', 'cat__Lead Origin_Landing Page Submission',
       'cat__Lead Origin_Lead Add Form', 'cat__Lead Origin_Lead Import',
       'cat__Lead Source_Click2call', 'cat__Lead Source_Direct Traffic',
       'cat__Lead Source_Facebook', 'cat__Lead Source_Google',
       'cat__Lead Source_Live Chat', 'cat__Lead Source_NC_EDM',
       'cat__Lead Source_Olark Chat', 'cat__Lead Source_Organic Search',
       'cat__Lead Source_Pay per Click Ads', 'cat__Lead Source_Reference',
       'cat__Lead Source_Referral Sites', 'cat__Lead Source_Social Media',
       'cat__Lead Source_WeLearn', 'cat__Lead Source_Welingak Webs

## Treinamento do Modelo de Voting Classifier

In [10]:
# Criar o modelo de VotingClassifier
lr_model = LogisticRegression(random_state=51)

# Para executar o modelo do Soft voting, é necessário colocar o hiperparâmetro probability = True no modelo SVC
svc_model = SVC(probability=True,kernel='linear')
tree_model = DecisionTreeClassifier(random_state=51)

voting_model = VotingClassifier(
    estimators=[
        ('logistic regression', lr_model),
        ('svc', svc_model),
        ('decision tree',tree_model)
    ],

    # Hard faz a votação pela maioria das predições dos estimadores
    # Soft faz a votação pela média ponderada das probabilidades de cada classe vindas de cada estimador
    voting='soft'
)


In [11]:
voting_model.fit(X_train,y_train)

,"estimators estimators: list of (str, estimator) tuplesInvoking the ``fit`` method on the ``VotingClassifier`` will fit clonesof those original estimators that will be stored in the class attribute``self.estimators_``. An estimator can be set to ``'drop'`` using:meth:`set_params`... versionchanged:: 0.21 ``'drop'`` is accepted. Using None was deprecated in 0.22 and support was removed in 0.24.","[('logistic regression', ...), ('svc', ...), ...]"
,"voting voting: {'hard', 'soft'}, default='hard'If 'hard', uses predicted class labels for majority rule voting.Else if 'soft', predicts the class label based on the argmax ofthe sums of the predicted probabilities, which is recommended foran ensemble of well-calibrated classifiers.",'soft'
,"weights weights: array-like of shape (n_classifiers,), default=NoneSequence of weights (`float` or `int`) to weight the occurrences ofpredicted class labels (`hard` voting) or class probabilitiesbefore averaging (`soft` voting). Uses uniform weights if `None`.",None
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for ``fit``.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionadded:: 0.18",None
,"flatten_transform flatten_transform: bool, default=TrueAffects shape of transform output only when voting='soft'If voting='soft' and flatten_transform=True, transform method returnsmatrix with shape (n_samples, n_classifiers * n_classes). Ifflatten_transform=False, it returns(n_classifiers, n_samples, n_classes).",True
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting will be printed as itis completed... versionadded:: 0.23",False
,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001


## Análise dos Resultados

In [12]:
y_pred = voting_model.predict(X_test)

In [13]:
y_pred

array([1, 0, 0, ..., 0, 0, 1], shape=(1815,))

In [14]:
# Calcular métricas
accuracy = accuracy_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
recall = recall_score(y_test,y_pred)
f1 = f1_score(y_test,y_pred)

In [15]:
print(f'Acurácia: {accuracy}')
print(f'Precisão: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')

Acurácia: 0.7862258953168044
Precisão: 0.7203125
Recall: 0.6880597014925374
F1-Score: 0.7038167938931298


In [16]:
# mostrar a matriz de confusao

conf_matrix = confusion_matrix(y_test,y_pred)

fig = px.imshow(conf_matrix,
                labels=dict(x='Predição', y='Real'),
                x=['Not Converted', 'Converted'],
                y=['Not Converted', 'Converted'],
                color_continuous_scale='Viridis' 
)

fig.update_traces(text=conf_matrix, texttemplate='%{z}')
fig.update_layout(coloraxis_showscale=False)

fig.show()

In [17]:
# Carregar as importâncias dos modelos
importances = []

for estimator in voting_model.estimators_:

    # Para modelos lineares, retorna coef
    if hasattr(estimator, 'coef_'):
        importances.append(np.abs(estimator.coef_[0]))
    # Para modelos baseados em árvores
    elif hasattr(estimator,'feature_importances_'):
        importances.append(estimator.feature_importances_)
    else:
        print(f'Não foi possível carregar as importâncias do modelo {type(estimator).__name__}')

        

In [18]:
importancia_media = np.mean(importances,axis=0)

importancia_media

array([0.22523852, 0.03587513, 0.11334446, 0.85206433, 0.08244887,
       0.0072784 , 0.03554339, 0.05729069, 0.04262539, 0.00572446,
       0.03675622, 0.01354635, 0.59017145, 0.62217065, 1.33160526,
       0.1270707 , 0.15406345, 0.332447  , 0.43163126, 0.12453701,
       0.01089549, 0.31246918, 0.32214603, 0.19972264, 0.04277403,
       0.27364251, 0.25843587, 0.35100332, 0.19591516, 0.88615977,
       0.1092695 , 0.33488185, 0.05459321, 0.14172053, 0.08172463,
       0.68320062, 0.60979761, 0.55129466, 0.02167269, 0.1209071 ,
       0.10067165, 0.29920279, 0.25233692, 0.6822895 , 0.48054617,
       0.31871189, 0.3549633 , 0.16075804, 0.22973273, 0.10262642,
       0.05419765, 0.02177073, 0.03318338, 0.18525745, 0.48984408,
       0.1209071 , 0.45366622, 0.1378337 , 0.05822045, 0.60502148,
       0.52612309, 0.57091876, 0.24872689, 0.35476814, 0.35041113,
       0.90905856, 0.02855717, 0.25441671])

In [19]:
feature_names = preprocessor.get_feature_names_out()
feature_names

array(['num__Do Not Email', 'num__Do Not Call', 'num__TotalVisits',
       'num__Total Time Spent on Website', 'num__Page Views Per Visit',
       'num__Search', 'num__Newspaper Article', 'num__X Education Forums',
       'num__Newspaper', 'num__Digital Advertisement',
       'num__Through Recommendations',
       'num__A free copy of Mastering The Interview',
       'cat__Lead Origin_API', 'cat__Lead Origin_Landing Page Submission',
       'cat__Lead Origin_Lead Add Form', 'cat__Lead Origin_Lead Import',
       'cat__Lead Source_Click2call', 'cat__Lead Source_Direct Traffic',
       'cat__Lead Source_Facebook', 'cat__Lead Source_Google',
       'cat__Lead Source_Live Chat', 'cat__Lead Source_NC_EDM',
       'cat__Lead Source_Olark Chat', 'cat__Lead Source_Organic Search',
       'cat__Lead Source_Pay per Click Ads', 'cat__Lead Source_Reference',
       'cat__Lead Source_Referral Sites', 'cat__Lead Source_Social Media',
       'cat__Lead Source_WeLearn', 'cat__Lead Source_Welingak Webs

In [20]:
df_feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importancia_media})

df_feature_importances

,Feature,Importance
0,num__Do Not Email,0.225239
1,num__Do Not Call,0.035875
2,num__TotalVisits,0.113344
3,num__Total Time Spent on Website,0.852064
4,num__Page Views Per Visit,0.082449
...,...,...
63,cat__Last Notable Activity_Resubscribed to emails,0.354768
64,cat__Last Notable Activity_SMS Sent,0.350411
65,cat__Last Notable Activity_Unreachable,0.909059
66,cat__Last Notable Activity_Unsubscribed,0.028557


In [21]:
df_feature_importances = df_feature_importances.sort_values(by='Importance', ascending=True)
df_feature_importances

,Feature,Importance
9,num__Digital Advertisement,0.005724
5,num__Search,0.007278
20,cat__Lead Source_Live Chat,0.010895
11,num__A free copy of Mastering The Interview,0.013546
38,cat__Last Activity_Email Link Clicked,0.021673
...,...,...
35,cat__Last Activity_Approached upfront,0.683201
3,num__Total Time Spent on Website,0.852064
29,cat__Lead Source_Welingak Website,0.886160
65,cat__Last Notable Activity_Unreachable,0.909059


In [22]:
fig = px.bar(df_feature_importances,
             x='Importance',
             y='Feature',
             orientation='h',
             title='Importância das Features (Voting Classifier)'
)

fig.update_layout(height=1280, width=1000)
fig.show()

## Propriedades do Modelo

In [25]:
X_test[7]

array([-0.28575686, -0.01660108,  0.49682342, -0.80365386,  0.29349295,
       -0.03895716, -0.01660108, -0.01173793, -0.01173793, -0.02033349,
       -0.02876185, -0.67670029,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ])

In [ ]:
# Mostrar evidências
# Selecionar um registro da base para fazer a predição

X_sample = X_test[7].reshape(1,-1)

# Predições individuais dos estimadores
log_pred = voting_model.named_estimators_['logistic regression'].predict(X_sample)
svc_pred = voting_model.named_estimators_['svc'].predict(X_sample)
tree_pred = voting_model.named_estimators_['decision tree'].predict(X_sample)

voting_pred = voting_model.predict(X_sample)


In [26]:
X_sample

array([[-0.28575686, -0.01660108,  0.49682342, -0.80365386,  0.29349295,
        -0.03895716, -0.01660108, -0.01173793, -0.01173793, -0.02033349,
        -0.02876185, -0.67670029,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ]])

In [24]:
print(f'Predição da Regressão Logística: {log_pred[0]}')
print(f'Predição do SVC: {svc_pred[0]}')
print(f'Predição da Árvore de Decisão: {tree_pred[0]}')
print(f'Predição do Voting model: {voting_pred[0]}')


Predição da Regressão Logística: 0
Predição do SVC: 0
Predição da Árvore de Decisão: 1
Predição do Voting model: 1


In [31]:
# Mostrar evidências
# Selecionar um registro da base para fazer a predição

X_sample = X_test[7].reshape(1,-1)

# Predições individuais dos estimadores
log_proba = voting_model.named_estimators_['logistic regression'].predict_proba(X_sample)
svc_proba = voting_model.named_estimators_['svc'].predict_proba(X_sample)
tree_proba = voting_model.named_estimators_['decision tree'].predict_proba(X_sample)

voting_proba = voting_model.predict_proba(X_sample)


In [32]:
print(f'Predição da Regressão Logística: {log_proba[0]}')
print(f'Predição do SVC: {svc_proba[0]}')
print(f'Predição da Árvore de Decisão: {tree_proba[0]}')
print(f'Predição do Voting model: {voting_proba[0]}')


Predição da Regressão Logística: [0.67050795 0.32949205]
Predição do SVC: [0.69534976 0.30465024]
Predição da Árvore de Decisão: [0. 1.]
Predição do Voting model: [0.4552859 0.5447141]
